In [1]:
import torch
from torch.utils.data import Dataset
import h5py
import numpy as np

class PCamDataset(Dataset):
    def __init__(self, h5_x_path, h5_y_path, transform=None):
        self.x_h5 = h5py.File(h5_x_path, 'r')
        self.y_h5 = h5py.File(h5_y_path, 'r')
        self.transform = transform
        self.length = len(self.y_h5['y'])

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        image = self.x_h5['x'][idx]  # shape: (96, 96, 3), dtype: uint8
        label = self.y_h5['y'][idx].item()


        # numpy → PIL → transform
        image = image.astype(np.uint8)
        image = transforms.ToPILImage()(image)

        if self.transform:
            image = self.transform(image)

        return image, label


In [2]:
from torchvision import transforms
from torch.utils.data import DataLoader

transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])





transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

train_dataset = PCamDataset('camelyonpatch_level_2_split_train_x.h5',
                             'camelyonpatch_level_2_split_train_y.h5',
                             transform=transform_train)

test_dataset = PCamDataset('camelyonpatch_level_2_split_test_x.h5',
                            'camelyonpatch_level_2_split_test_y.h5',
                            transform=transform_test)
# Validation dataset
val_dataset = PCamDataset('valid_x_uncompressed.h5', 'valid_y_uncompressed.h5', transform=transform_test)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


In [3]:
import torch.nn as nn
from torchvision.models import resnet50, ResNet50_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.fc.in_features, 2)
)

model = model.to(device)


In [4]:
import h5py
import numpy as np

with h5py.File('camelyonpatch_level_2_split_train_y.h5', 'r') as f:
    labels = f['y'][:]
    unique, counts = np.unique(labels, return_counts=True)
    print(dict(zip(unique, counts)))


{np.uint8(0): np.int64(131072), np.uint8(1): np.int64(131072)}


In [5]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-4)
num_epochs = 30
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', patience=3, factor=0.5)

for epoch in range(num_epochs):
    model.train()
    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    # テスト精度評価
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1} Test Accuracy: {acc*100:.2f}%")

    
    scheduler.step(acc)



Epoch 1 Test Accuracy: 86.38%


KeyboardInterrupt: 

In [ ]:
from torchvision.models import resnet50, ResNet50_Weights
import torch.nn as nn
import torch

# 1. モデル構造を再定義（学習時と同じ）
model = resnet50(weights=ResNet50_Weights.DEFAULT)
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 2)
)

# 2. GPU使用
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 3. ✅ ここでベストモデルを読み込む！
model.load_state_dict(torch.load("best_model_epoch2.pth"))
model.eval()


tta_transforms = [
    transforms.Compose([
        transforms.ToPILImage(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomHorizontalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomVerticalFlip(p=1.0),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
]

def tta_predict(model, image_tensor, tta_transforms):
    model.eval()
    preds = []
    image_np = (image_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8)

    for transform in tta_transforms:
        aug_img = transform(image_np)  # PIL処理を含む
        aug_img = aug_img.unsqueeze(0).to(device)

        with torch.no_grad():
            output = model(aug_img)
            prob = torch.softmax(output, dim=1)
            preds.append(prob.cpu())

    mean_pred = torch.mean(torch.stack(preds), dim=0)
    return torch.argmax(mean_pred, dim=1).item()


/tmp/ipykernel_214979/2313879289.py:17: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model_epoch2.pth"))


In [ ]:
from torchvision import transforms

to_tensor = transforms.Compose([
    transforms.ToTensor()
])

for i in range(len(val_dataset)):
    img, label = val_dataset[i]  # img: PIL.Image
    img = to_tensor(img)         # Tensor化
    pred = tta_predict(model, img, tta_transforms)

    if pred == label:
        correct += 1
    total += 1


print(f"TTA Validation Accuracy: {100 * correct / total:.2f}%")


TTA Validation Accuracy: 87.10%
